In [1]:
import os
%run funciones.py
import json
import pandas as pd
import numpy as np
from funciones import *
from keras.models import load_model

In [2]:
pruebas = pd.read_csv("..\data\pruebas.csv")

stop = list(pruebas['name stops'])
real = list(pruebas['real'])
link = list(zip(pruebas['ini_link'], pruebas['end_link']))
time = list(zip(pruebas['hora'], pruebas['minuto']))

c = ['Nro','Stops','Link', 'Time', 'Real', 'Mean_Set', 'err_mean_set', 'input_set', 'err_input_set', 'input_concat', 'err_input_concat', 'model']
preds = pd.DataFrame(columns=c)

for file in os.listdir('../models/'):
    model_path = f'../models/{file}'
    json_path = f'../info_models/{file[:-2]}json'

    # cargar archivos
    info_model = json.load(open(json_path))
    model = load_model(model_path, compile=False)

    # ingresar parametros
    div_split = info_model['div_split']
    input_out_length = (info_model['INPUT_LENGTH'], info_model['OUTPUT_LENGTH'])  
    SCALER = info_model['scaler']
    CONV = info_model['isCONV']
    range_links = tuple(info_model['range_links'])
    path_dataset = "../data/"+info_model['name_dataset']
    input_shape = (1, ) + model.input_shape[1:]
    print(f"{file}")
    print(f"Parametros de prediccion: {div_split}, {input_out_length}, {SCALER}, CONV:{CONV}, {range_links}, {path_dataset}, {input_shape}")

    # leer dataset
    cols = ['id_linkref', 'horas', 'minute', 'travel_time']
    origin_dataset = pd.read_csv(path_dataset, sep=',', usecols=cols)
    dataset = origin_dataset[cols]

    # delimitar links
    if range_links != (0, 82):
        dataset =  dataset[(dataset['id_linkref']>= range_links[0]) & (dataset['id_linkref']<=range_links[1])]
    print(f"Links del dataset desde {range_links[0]} - {range_links[1]} = {len(dataset['id_linkref'].unique())} links")

    # preparar dataset para la prediccion segun el modelo
    data, scaler = cargar_dataset(dataset, div_split, input_out_length, SCALER, CONV)

    dt_input = data['ts'].reset_index()
    for i in range(len(real)):
        # encontrar input: retorna indices de los samples que coniden como input
        res = find_input(dt_input, stop=link[i][0], time=time[i], input_size=input_out_length[0])

        # escoger un indice aleatorio para pa prediccion
        tmp = np.random.choice(len(res))

        # predecir el tiempo de viaje
        mean, y_of_set, y_of_pred = predict_travel_time(
            n_links = link[i][1] - link[i][0],
            shape_input= input_shape,
            id_sample= res[tmp],
            data_in_s= data['x_ts_s'],
            data_in = data['x_ts'],
            modelo = model,
            scaler=scaler,
            conv=CONV
        )

        # calcular tiempo de viaje, suma de todas las predicciones en cada link
        p_mean = round(sum(mean),0) # promedio, para verificar en que se basa las predicciones
        p_set = round(sum(y_of_set), 0) # usamos input un grupo de muestras del dataset test
        p_concat = round(sum(y_of_pred), 0) # concadenamos las predicciones al siguiente input

        # calcular el error de cada forma de prediccion
        err_p_mean = real[i] - p_mean
        err_p_set = real[i] - p_set
        err_p_concat = real[i] - p_concat 

        # agregar al df pred
        preds.loc[len(preds)] = [i+1,stop[i], link[i], time[i], real[i], p_mean, err_p_mean, p_set, err_p_set, p_concat, err_p_concat, file[:-3]]

    print('=> Pruebas Completadas')
    print('='*120)

ConvLSTMEnco_Robus_e150b256_80e.h5
Parametros de prediccion: 80, (40, 1), Robust, CONV:True, (2, 81), ../data/travel_time_series_2col.csv, (1, 40, 4, 1, 1)
Links del dataset desde 2 - 81 = 80 links
=> Pruebas Completadas
ConvLSTMEnco_Robus_epo150_batch256-36.h5
Parametros de prediccion: 36, (36, 1), Robust, CONV:True, (14, 49), ../data/travel_time_series_2col.csv, (1, 36, 4, 1, 1)
Links del dataset desde 14 - 49 = 36 links
=> Pruebas Completadas
LSMT2_e100b256_Robust_in36-4_dt14-49.h5
Parametros de prediccion: 36, (36, 1), Robust, CONV:False, (14, 49), ../data/travel_time_series_2col.csv, (1, 36, 4)
Links del dataset desde 14 - 49 = 36 links
=> Pruebas Completadas
LSMT2_e100b256_Robust_in40-4_dt0-82.h5
Parametros de prediccion: 40, (40, 1), Robust, CONV:False, (0, 82), ../data/travel_time_series_2col.csv, (1, 40, 4)
Links del dataset desde 0 - 82 = 83 links
=> Pruebas Completadas


In [3]:
preds

,Nro,Stops,Link,Time,Real,Mean_Set,err_mean_set,input_set,err_input_set,input_concat,err_input_concat,model
0,1,Primer Paradero - Hospital,"(39, 45)","(10, 30)",379,321.0,58.0,510.0,-131.0,510.0,-131.0,ConvLSTMEnco_Robus_e150b256_80e
1,2,Primer Paradero - UNSAAC,"(39, 46)","(10, 30)",457,529.0,-72.0,634.0,-177.0,633.0,-176.0,ConvLSTMEnco_Robus_e150b256_80e
2,3,Primer Paradero - Hospital,"(39, 45)","(16, 45)",397,536.0,-139.0,450.0,-53.0,449.0,-52.0,ConvLSTMEnco_Robus_e150b256_80e
3,4,Primer Paradero - UNSAAC,"(39, 46)","(16, 45)",513,623.0,-110.0,556.0,-43.0,556.0,-43.0,ConvLSTMEnco_Robus_e150b256_80e
4,1,Primer Paradero - Hospital,"(39, 45)","(10, 30)",379,499.0,-120.0,365.0,14.0,365.0,14.0,ConvLSTMEnco_Robus_epo150_batch256-36
5,2,Primer Paradero - UNSAAC,"(39, 46)","(10, 30)",457,605.0,-148.0,464.0,-7.0,464.0,-7.0,ConvLSTMEnco_Robus_epo150_batch256-36
6,3,Primer Paradero - Hospital,"(39, 45)","(16, 45)",397,374.0,23.0,421.0,-24.0,421.0,-24.0,ConvLSTMEnco_Robus_epo150_batch256-36
7,4,Primer Paradero - UNSAAC,"(39, 46)","(16, 45)",513,489.0,24.0,519.0,-6.0,519.0,-6.0,ConvLSTMEnco_Robus_epo150_batch256-36
8,1,Primer Paradero - Hospital,"(39, 45)","(10, 30)",379,372.0,7.0,417.0,-38.0,394.0,-15.0,LSMT2_e100b256_Robust_in36-4_dt14-49
9,2,Primer Paradero - UNSAAC,"(39, 46)","(10, 30)",457,516.0,-59.0,426.0,31.0,466.0,-9.0,LSMT2_e100b256_Robust_in36-4_dt14-49


In [12]:
total_err_input_set = {}
total_err_input_concat = {}
for mod in preds['model'].unique():
    df_tmp = preds[preds['model'] == mod]
    total_err_input_set[mod] = abs(df_tmp['err_input_set']).sum()
    total_err_input_concat[mod] = abs(df_tmp['err_input_concat']).sum()

In [15]:
total_err_input_set

{'ConvLSTMEnco_Robus_e150b256_80e': 404.0,
 'ConvLSTMEnco_Robus_epo150_batch256-36': 51.0,
 'LSMT2_e100b256_Robust_in36-4_dt14-49': 113.0,
 'LSMT2_e100b256_Robust_in40-4_dt0-82': 90.0}

In [16]:
total_err_input_concat

{'ConvLSTMEnco_Robus_e150b256_80e': 402.0,
 'ConvLSTMEnco_Robus_epo150_batch256-36': 51.0,
 'LSMT2_e100b256_Robust_in36-4_dt14-49': 67.0,
 'LSMT2_e100b256_Robust_in40-4_dt0-82': 97.0}

In [ ]:
# leer dataset busstop
df_stops = pd.read_csv('../data/busstops_ida.csv')

# df_stops.info()
def generateLinks(row):
    row['end_stop'] = df_linkref.loc[row.name+1, 'id']
    row['name_end_stop'] = df_linkref.loc[row.name+1, 'name']
    row['linkref'] = str(row['id']) + ':' + str(df_linkref.loc[row.name+1, 'id'])
    return row

df_linkref = df_stops.copy()
df_linkref = df_linkref.head(df_linkref.shape[0]-1).apply(generateLinks, axis=1)
df_linkref = df_linkref.drop(df_linkref.columns[2:5], axis=1)
df_linkref = df_linkref.rename(columns={'id':'init_stop'})
df_linkref['end_stop'] = df_linkref['end_stop'].values.astype(str)
print(df_linkref.to_string())

    number  init_stop  longitude  radio end_stop              name_end_stop  linkref
0        0          0 -71.856025     30       36                primer stop     0:36
1        1         36 -71.857300     18       37               segundo stop    36:37
2        2         37 -71.861944     13       38                   Ccollana    37:38
3        3         38 -71.867299      9       39            Puente Huaccoto    38:39
4        4         39 -71.870211     10       40                      Kayra    39:40
5        5         40 -71.873963     10       41                   Forestal    40:41
6        6         41 -71.876104      8       42                Grifo Tapia    41:42
7        7         42 -71.878642     11       43                      Posta    42:43
8        8         43 -71.880031     15       44                  Romeritos    43:44
9        9         44 -71.880933      7       45                    Tercero    44:45
10      10         45 -71.882989      7       46                 

In [ ]:
45-50, 10:30

df = fn (45 10 30)  8


df_test

0
1
2

45 
...

81
82
83= 
1